In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

from sktime.forecasting.arima import ARIMA

import DataRetriever as dr

retriever = dr.DataRetriever()
year1 = retriever.get_data("All-Subsystems-hour-Year1.pkl")
year2 = retriever.get_data("All-Subsystems-hour-Year2.pkl")

c:\users\magnu\appdata\local\programs\python\python39\lib\site-packages\sktime\utils\validation\series.py:21: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
c:\users\magnu\appdata\local\programs\python\python39\lib\site-packages\sktime\forecasting\base\_fh.py:18: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  RELATIVE_TYPES = (pd.Int64Index, pd.RangeIndex)
c:\users\magnu\appdata\local\programs\python\python39\lib\site-packages\sktime\forecasting\base\_fh.py:19: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  ABSOLUTE_TYPES = (pd.Int64Index, pd.RangeIndex, pd.DatetimeIndex, pd.PeriodInd

In [2]:
gen_year2 = year2[year2["PV_Watts3PhTotalW3PhT1"].isna() == False]
gen_year2 = gen_year2[gen_year2["PV_Watts3PhTotalW3PhT2"].isna() == False] #There are 87 rows with NaN values.

print(f"A total of {len(year2) - len(gen_year2)} rows have been dropped since they have NaN values.")

gen_year2["Generated Energy"] = (gen_year2["PV_Watts3PhTotalW3PhT1"].to_numpy() + gen_year2["PV_Watts3PhTotalW3PhT2"].to_numpy())
gen_year2 = gen_year2[["Generated Energy"]] / 1e3 #Convert Wh to kWh
gen_year2

A total of 87 rows have been dropped since they have NaN values.


,Generated Energy
Timestamp,
2015-02-01 00:00:00,0.018496
2015-02-01 01:00:00,0.018332
2015-02-01 02:00:00,0.018385
2015-02-01 03:00:00,0.018502
2015-02-01 04:00:00,0.018524
...,...
2016-01-31 19:00:00,0.017710
2016-01-31 20:00:00,0.017940
2016-01-31 21:00:00,0.018149


In [3]:
gen_df = gen_year2

gen_df = gen_df.resample('d').sum()
gen_df

,Generated Energy
Timestamp,
2015-02-01,12.958986
2015-02-02,0.010302
2015-02-03,41.425352
2015-02-04,40.553641
2015-02-05,42.718180
...,...
2016-01-27,33.832924
2016-01-28,43.440532
2016-01-29,27.944539


In [7]:
from sktime.forecasting.arima import AutoARIMA

forecaster = AutoARIMA(sp = 1, suppress_warnings=True)
forecaster.fit(gen_df)

AutoARIMA(suppress_warnings=True)

In [8]:
y_pred = forecaster.predict(fh=list(range(1, 500)))
y_pred

c:\users\magnu\appdata\local\programs\python\python39\lib\site-packages\sktime\forecasting\base\_fh.py:100: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  values = pd.Int64Index(values, dtype=int)
c:\users\magnu\appdata\local\programs\python\python39\lib\site-packages\sktime\forecasting\base\_fh.py:337: FutureWarning: Timestamp.freqstr is deprecated and will be removed in a future version.
  cutoff = _coerce_to_period(cutoff, freq=cutoff.freqstr)


,Generated Energy
2016-02-01,28.590267
2016-02-02,25.648607
2016-02-03,24.926055
2016-02-04,24.748577
2016-02-05,24.704983
...,...
2017-06-09,24.690789
2017-06-10,24.690789
2017-06-11,24.690789
2017-06-12,24.690789


In [9]:
forecaster.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  365
Model:               SARIMAX(1, 1, 1)   Log Likelihood               -1555.424
Date:                Wed, 30 Mar 2022   AIC                           3116.848
Time:                        13:40:31   BIC                           3128.539
Sample:                             0   HQIC                          3121.495
                                - 365                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2456      0.055      4.453      0.000       0.138       0.354
ma.L1         -0.9368      0.020    -46.014      0.000      -0.977      -0.897
sigma2       300.0345     26.776     11.205      0.000     247.555     352.514
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 6.04
Prob(Q):                              0.96   Prob(JB):                         0.05
Heteroskedasticity (H):               0.60   Skew:                            -0.17
Prob(H) (two-sided):                  0.01   Kurtosis:                         2.47
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""